In [ ]:
import os, sys
import numpy as np 
import gmat_nav as gnav
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
# Directory location of this Jupyter notebook tutorial
file_dir = os.path.abspath('') #os.path.dirname(os.path.abspath(__file__))
# --------------- USER DEFINED PATHS (USER -- PLEASE LOOK HERE!) ------------------# 
gmat_root_dir = '/home/natsubuntu/Desktop/SysControl/estimation/CauchyCPU/CauchyEst_Nat/GMAT'
gmat_data_dir = file_dir + "/data"
eop_filepath = gmat_data_dir + "/eop_file.txt"
spaceweather_filepath = gmat_data_dir + "/SpaceWeather-v1.2.txt"
gnav.check_then_load_gmatpy(gmat_root_dir, eop_filepath, spaceweather_filepath)
gmat = gnav.gmat # our handle to the gmat module, loaded through gnav
gmat.Clear() # clears any junk lying around 

# Our Satellite Parameters
# Remember, your intial epoch time must be UTC Gregorian, otherwise use time_convert(...) function in gnav to switch it!
t0 = datetime(year=2023, month=7, day=10, hour=19, minute=34, second=54, microsecond=0)
pos3 = np.array([4.9962452882705193e+03,3.8779464630861030e+03,2.7360432364171807e+03]) # approx a 550 (units -> kilometer) orbit
vel3 = np.array([-5.0280935744461930e+00,5.5759213419992673e+00,1.2698611722905329e+00]) # speed (units -> kilometers/second) 
x0 = np.concatenate((pos3, vel3)) # Your initial state vector
Cd0 = 2.1 # nominal coefficient of drag # unitless
Cr0 = 0.75 # nominal coefficient of reflectivity # unitless
A = 14.18 # (Drag) Area of the satellite # units -> meters
m = 3995.6 # (dry) mass of the satellite # units -> kilograms
fuel_mass = 359.9 # units -> kilograms
solar_flux = 1370.052 # units -> Watts/m^2
dt = 60.0 # units -> seconds (Our Time Step)

Welcome to tutorial 4! You've braved it so far, lets put everything together and build an EKF for Orbit Determination using GMAT.

Most pieces required to simulate the EKF using GMAT have already been introduced (see tutorials one to three). Lets start by running an EKF simulation with a "really good" GPS point solution as its measurement. The better the measurement, the better you will track your solve for states. Solve for states are generally hard to observe, requiring long estimation horizons to track. We can show, however, that by simulating a "really good" GPS point solution, you can track your solve fors very well. This, at the very least, shows that the partial derivative information returned in the jacobian (and used by the EKF via the STM), is accurate. We'll simulate realistic GPS point solutions as measurements afterwards, where you'll then see that the EKF has a much harder time tracking solve-fors.

Let get started! Below shows the EKF for a "really good" GPS point solution (one with very little noise). The standard deviation for the GPS point solution is set to 7.5 micrometers...quite unrealistic.

In [ ]:
# -- Comment for different realizations each time
np.random.seed(10)
# -- For re-running cell... destroy GMAT and then recreate satellite object
gmat.Clear() 
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)

# -- Add a solve for state for Cd, as a FOGM model
sf_scale = 0.0013 # standard deviation for changes in Cd, per dt
sf_tau = 21600 # half life of 6 hours
sat.set_solve_for(field = "Cd", dist="gauss", scale = sf_scale, tau = sf_tau) # Adds a seventh state

# -- Simulate satellite trajectory
num_steps = 500 # number of simulation steps
gps_std_dev = 0.0000000075 # kilometers, std dev is 7.5 micrometers, for each of the three (x,y,z) direction
xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=None) # returns states, simulated gps measurements (in earth body frame), process noise samples for the realization, and measurement noise samples from the realization.
# Do note the seventh state here is our solve-for state

# -- Construct Kalman Filter Noise Covariances
# Process Noise
w_PSD = np.repeat(1e-23, 3) # The power spectral density for acceleration noises in (x,y,z) # Very little process noise exists in pos/vel for a well specified model
W7 = np.zeros((7,7))
W7[0:6,0:6] = gnav.leo6_process_noise_model(dt, w_PSD)
W7[6,6] = sf_scale**2 # Variance of our solve for state
# Measurement Noise
V3 = np.diag(np.ones(3)) * gps_std_dev**2

# -- Construct Starting State and Covariance of EKF
P_kf = np.diag(np.ones(7)) * (0.001)**2 # 1 m std dev, 1 m/s std dev, .001 solve for std dev
x_kf = np.random.multivariate_normal(xs[0], P_kf)
x_kf[6] = np.clip(x_kf[6], -0.999, np.inf) # Keep solve for state in range (-1, np.inf)
Ps_kf = [P_kf.copy()] 
xs_kf = [x_kf.copy()] 
# -- Estimation Constants
STM_power_series_order = 3
num_STM_sub_steps_per_dt = 2
with_STM_avg_jac = True # average the jacobian over the endpoints of the step (or substep)
I7 = np.eye(7)

# -- Begin Estimation Loop
# Reset satellite state to the initial estimate 
N = xs.shape[0]
sat.reset_state(x_kf)
for k in range(1,N): # The initial measurement is for the initial state x0, we run TP first, so take the second GPS msmt
    zk = zs[k]
    # Time Propagation (TP)
    Phik = sat.get_transition_matrix(STM_power_series_order, num_STM_sub_steps_per_dt, with_STM_avg_jac)
    P_kf = Phik @ P_kf @ Phik.T + W7 
    x_kf = sat.step()
    # Measurement Update (MU)
    zk_bar = sat.transform_Earth_MJ2000Eq_2_BodyFixed()[0:3] # Transform current state from EarthMJ2000Eq to Earth Body Fixed (Frame of GPS measurements)
    resid = zk - zk_bar
    Hk = sat.get_last_position_rotation_matrix()
    Kk = np.linalg.solve(Hk @ P_kf @ Hk.T + V3, Hk @ P_kf).T # which is a numerically more stable -> P_kf @ Hk.T @ np.linalg.inv(Hk @ P_kf @ Hk.T + V3)
    x_kf = x_kf + Kk @ resid # State Update 
    P_kf = (I7 - Kk @ Hk) @ P_kf @ (I7 - Kk @ Hk).T + Kk @ V3 @ Kk.T # Covariance Update -> Joseph Formulation -> Keeps P_kf symmetric
    # Reset Satellite State
    sat.get_ellapsed_time()
    sat.reset_state(x_kf)
    # Append to State/Covariance Lists
    xs_kf.append(x_kf.copy())
    Ps_kf.append(P_kf.copy())
# Convert Lists to Numpy Arrays 
xs_kf = np.array(xs_kf)
Ps_kf = np.array(Ps_kf)

# Plot the EKF Using a State Error Plot
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator
gnav.plot_kf_errors_against_sigma_bound(xs, xs_kf, Ps_kf, sigma_bound, dt, solve_for_labels=["Solve For\nCd"])

It may be hard to view the results as we started with a large covariance, relative to the steady state solution. Chopping off the first few time steps will fix this:  Lets also view the tracking of $C_d$ explicitly:

In [ ]:
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator
gnav.plot_kf_errors_against_sigma_bound(xs[10:], xs_kf[10:], Ps_kf[10:], sigma_bound, dt, solve_for_labels=["Solve For Cd"])

# Show Cd Truth vs Cd Estimate
Ts = np.arange(xs.shape[0]) * dt / 3600
plt.figure()
plt.title("Cd Truth (Red) and Cd Estimate (Blue) against Time (hours)")
plt.plot(Ts, Cd0*(1+xs[:,6]), 'r')
plt.plot(Ts, Cd0*(1+xs_kf[:,6]), 'b')
#plt.plot(Ts, xs[:,6], 'r')
#plt.plot(Ts, xs_kf[:,6], 'b')
plt.ylabel("Cd")
plt.xlabel("Time (hours)")
plt.show()

As we can see above, the EKF is doing what we'd expect: Its tracking changes in Cd very well. Position and velocity are tracked near perfect. If you wish, add $C_r$ as a solve for state above. You will see that even with such favorable noise settings, estimating $C_r$'s solve for is still challenging. 

The previous simulation, of course, is not realistic. For starters, the GPS noise standard deviation was set to 7.5 micrometers: a near perfect measurement. In reality, this is more like 7.5 meters. The Power Spectral Density of acceleration noise was also set extremely small, leading to a position uncertainty (standard deviation) of $8.485e$-$7$ meters over a propagation step of 60 seconds, and a velocity uncertainty (standard deviation) of $2.49e$-$8$ meters/sec over a propagation interval of 60 seconds. This implies we are extremely confident in our model. In practice, a slight perturbation or modelling error could lead to filter divergence with such a tiny process noise (you can try this yourself: generate measurements with one satellite model, and use a similar but different model for the EKF). 

Let's move on to a more realistic simulation, where the GPS noise standard deviation is instead 7.5 meters. We will also turn the filter's process noise up (position uncertainty of 0.268 meters per propagation of dt=60 sec, and velocity uncertainty of 7.75 millimeters/second per propagation of dt=60sec). Doing so tells the filter we are much less confident (than before) in our model. The code below is a copy and paste of that above, with only the measurement + process noise changed as suggested. Give it a run:

In [ ]:
# -- Comment for different realizations each time
np.random.seed(10)
# -- For re-running cell... destroy GMAT and then recreate satellite object
gmat.Clear() 
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)

# -- Add a solve for state for Cd, as a FOGM model
sf_scale = 0.0013 # standard deviation for changes in Cd, per dt
sf_tau = 21600 # half life of 6 hours
sat.set_solve_for(field = "Cd", dist="gauss", scale = sf_scale, tau = sf_tau) # Adds a seventh state

# -- Simulate satellite trajectory
num_steps = 500 # number of simulation steps
gps_std_dev = 0.0075 # kilometers, std dev is 7.5 meters, for each of the three (x,y,z) direction
xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=None) # returns states, simulated gps measurements (in earth body frame), process noise samples for the realization, and measurement noise samples from the realization.
# Do note the seventh state here is our solve-for state

# -- Construct Kalman Filter Noise Covariances
# Process Noise
w_PSD = np.repeat(1e-12, 3) # The power spectral density for acceleration noises in (x,y,z) # Very little process noise exists in pos/vel for a well specified model
W7 = np.zeros((7,7))
W7[0:6,0:6] = gnav.leo6_process_noise_model(dt, w_PSD)
W7[6,6] = sf_scale**2 # Variance of our solve for state
# Measurement Noise
V3 = np.diag(np.ones(3)) * gps_std_dev**2

# -- Construct Starting State and Covariance of EKF
P_kf = np.diag(np.ones(7)) * (0.001)**2 # 1 m std dev, 1 m/s std dev, .001 solve for std dev
x_kf = np.random.multivariate_normal(xs[0], P_kf)
x_kf[6] = np.clip(x_kf[6], -0.999, np.inf) # Keep solve for state in range (-1, np.inf)
Ps_kf = [P_kf.copy()] 
xs_kf = [x_kf.copy()] 
# -- Estimation Constants
STM_power_series_order = 3
num_STM_sub_steps_per_dt = 2
with_STM_avg_jac = True # average the jacobian over the endpoints of the step (or substep)
I7 = np.eye(7)

# -- Begin Estimation Loop
# Reset satellite state to the initial estimate 
N = xs.shape[0]
sat.reset_state(x_kf)
for k in range(1,N): # The initial measurement is for the initial state x0, we run TP first, so take the second GPS msmt
    zk = zs[k]
    # Time Propagation (TP)
    Phik = sat.get_transition_matrix(STM_power_series_order, num_STM_sub_steps_per_dt, with_STM_avg_jac)
    P_kf = Phik @ P_kf @ Phik.T + W7 
    x_kf = sat.step()
    # Measurement Update (MU)
    zk_bar = sat.transform_Earth_MJ2000Eq_2_BodyFixed()[0:3] # Transform current state from EarthMJ2000Eq to Earth Body Fixed (Frame of GPS measurements)
    resid = zk - zk_bar
    Hk = sat.get_last_position_rotation_matrix()
    Kk = np.linalg.solve(Hk @ P_kf @ Hk.T + V3, Hk @ P_kf).T # which is a numerically more stable -> P_kf @ Hk.T @ np.linalg.inv(Hk @ P_kf @ Hk.T + V3)
    x_kf = x_kf + Kk @ resid # State Update 
    P_kf = (I7 - Kk @ Hk) @ P_kf @ (I7 - Kk @ Hk).T + Kk @ V3 @ Kk.T # Covariance Update -> Joseph Formulation -> Keeps P_kf symmetric
    # Reset Satellite State
    sat.get_ellapsed_time()
    sat.reset_state(x_kf)
    # Append to State/Covariance Lists
    xs_kf.append(x_kf.copy())
    Ps_kf.append(P_kf.copy())
# Convert Lists to Numpy Arrays 
xs_kf = np.array(xs_kf)
Ps_kf = np.array(Ps_kf)

# Plot the EKF Using a State Error Plot
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator
gnav.plot_kf_errors_against_sigma_bound(xs, xs_kf, Ps_kf, sigma_bound, dt, solve_for_labels=["Solve For\nCd"])

Removing the initial few time steps again helps better visualize the result

In [ ]:
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator
gnav.plot_kf_errors_against_sigma_bound(xs[10:], xs_kf[10:], Ps_kf[10:], sigma_bound, dt, solve_for_labels=["Solve For Cd"])

# Show Cd Truth vs Cd Estimate
Ts = np.arange(xs.shape[0]) * dt / 3600
plt.figure()
plt.title("Cd Truth (Red) and Cd Estimate (Blue) against Time (hours)")
plt.plot(Ts, Cd0*(1+xs[:,6]), 'r')
plt.plot(Ts, Cd0*(1+xs_kf[:,6]), 'b')
#plt.plot(Ts, xs[:,6], 'r')
#plt.plot(Ts, xs_kf[:,6], 'b')
plt.ylabel("Cd")
plt.xlabel("Time (hours)")
plt.show()

The most obvious difference is that $C_d$'s solve for state is estimated much more poorly than before. Moreover, estimation errors as large as 10 meters are seen in position. Lowering the process noise will result in better estimation, somewhat obviously, as we do have a perfect model. Velocity is still estimated very accurately. 

Lets now make a realistic simulation that abides by the noise parameters used in the Fermi satellite. We will add one kicker: non-Gaussian changes in atmospheric density, which we will simulate through non-Gaussian changes to $C_d$. The following code changes 

> 1.) The distribution forcing $C_d$'s solve for state. The $\alpha$ used is 1.3, consistent with space weather analysis. Revisit the end of tutorial 3 if this is unclear.

<br>

> 2.) The GPS measurement noise standard deviation is set to 7.5 meters

<br>

> 3.) The process noise PSD is set to 1e-15, a common choice for accurately modelled satellites. 

<br>

> 4.) Process noise is added to position and velocity in the simulate() function. This is a contetious thing to do in some Orbital Determination social circles. It will, however, show us if our model is robust to slight perturbation.

One notable mention is that since $C_d$ is now forced by a heavy-tailed distribution, we may need to scale-up its standard deviation in the filter's process noise model. This is an ad-hoc choice. If a large impulse is realized, the process noise levels may not cut it, and the filter may diverge slightly (or a lot). Boosting position and velocity's process noise value can also help. However, doing so trades-off estimation accuracy for robustness to outliers. 

The seed is set such that a large change in $C_d$ is observed. You may need to change the seed on your computer, if you do not see a large jump.

In [ ]:
# -- Comment for different realizations each time
np.random.seed(10)
#throw away some random samples
junk = np.random.randn(3000)

# some variables for this code-block
sf_scale = 0.0013 # standard deviation for changes in Cd, per dt
sf_tau = 21600 # half life of 6 hours
sf_alpha = 1.3 # alpha value for changes to Cd
num_steps = 1000 # number of simulation steps
gps_std_dev = 0.0075 # kilometers, std dev is 7.5 meters, for each of the three (x,y,z) direction
w_PSD = np.repeat(1e-15, 3) # The power spectral density for acceleration noises in (x,y,z) # Very little process noise exists in pos/vel for a well specified model
sf_boost = 1 # Hueristic! You can change!
STM_power_series_order = 3
num_STM_sub_steps_per_dt = 2
with_STM_avg_jac = True # average the jacobian over the endpoints of the step (or substep)
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator

# -- For re-running cell... destroy GMAT and then recreate satellite object
gmat.Clear() 
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)

# -- Add a solve for state for Cd, as a FOGM model
sat.set_solve_for(field = "Cd", dist="sas", scale = sf_scale, tau = sf_tau, alpha=sf_alpha) # Adds a seventh state
#sat.set_solve_for(field = "Cd", dist="gauss", scale = sf_scale, tau = sf_tau) # Adds a seventh state

# -- Simulate satellite trajectory
W6 = gnav.leo6_process_noise_model(dt, w_PSD)
xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=0*W6) # returns states, simulated gps measurements (in earth body frame), process noise samples for the realization, and measurement noise samples from the realization.
# Do note the seventh state here is our solve-for state

# -- Construct Kalman Filter Noise Covariances
# Process Noise
W7 = np.zeros((7,7))
W7[0:6,0:6] = W6
W7[6,6] = sf_boost * sf_scale**2 # Variance of our solve for state
# Measurement Noise
V3 = np.diag(np.ones(3)) * gps_std_dev**2

# -- Construct Starting State and Covariance of EKF
P_kf = np.diag(np.ones(7)) * (0.001)**2 # 1 m std dev, 1 m/s std dev, .001 solve for std dev
x_kf = np.random.multivariate_normal(xs[0], P_kf)
x_kf[6] = np.clip(x_kf[6], -0.999, np.inf) # Keep solve for state in range (-1, np.inf)
Ps_kf = [P_kf.copy()] 
xs_kf = [x_kf.copy()] 
I7 = np.eye(7)

# -- Begin Estimation Loop
# Reset satellite state to the initial estimate 
N = xs.shape[0]
sat.reset_state(x_kf)
for k in range(1,N): # The initial measurement is for the initial state x0, we run TP first, so take the second GPS msmt
    zk = zs[k]
    # Time Propagation (TP)
    Phik = sat.get_transition_matrix(STM_power_series_order, num_STM_sub_steps_per_dt, with_STM_avg_jac)
    P_kf = Phik @ P_kf @ Phik.T + W7 
    x_kf = sat.step()
    # Measurement Update (MU)
    zk_bar = sat.transform_Earth_MJ2000Eq_2_BodyFixed()[0:3] # Transform current state from EarthMJ2000Eq to Earth Body Fixed (Frame of GPS measurements)
    resid = zk - zk_bar
    Hk = sat.get_last_position_rotation_matrix()
    Kk = np.linalg.solve(Hk @ P_kf @ Hk.T + V3, Hk @ P_kf).T # which is a numerically more stable -> P_kf @ Hk.T @ np.linalg.inv(Hk @ P_kf @ Hk.T + V3)
    x_kf = x_kf + Kk @ resid # State Update 
    P_kf = (I7 - Kk @ Hk) @ P_kf @ (I7 - Kk @ Hk).T + Kk @ V3 @ Kk.T # Covariance Update -> Joseph Formulation -> Keeps P_kf symmetric
    # Reset Satellite State
    sat.get_ellapsed_time()
    sat.reset_state(x_kf)
    # Append to State/Covariance Lists
    xs_kf.append(x_kf.copy())
    Ps_kf.append(P_kf.copy())
# Convert Lists to Numpy Arrays 
xs_kf = np.array(xs_kf)
Ps_kf = np.array(Ps_kf)

# Plot the EKF Using a State Error Plot
gnav.plot_kf_errors_against_sigma_bound(xs, xs_kf, Ps_kf, sigma_bound, dt, solve_for_labels=["Solve For\nCd"])

As done before, lets zoom in a bit, and look at the estimation performance for tracking $C_d$.

In [ ]:
sigma_bound = 1 # One Sigma Bound: 68% of time the estimate should be inside, for a well tuned estimator
gnav.plot_kf_errors_against_sigma_bound(xs[10:], xs_kf[10:], Ps_kf[10:], sigma_bound, dt, solve_for_labels=["Solve For Cd"])

# Show Cd Truth vs Cd Estimate
Ts = np.arange(xs.shape[0]) * dt / 3600
plt.figure()
plt.title("Cd Truth (Red) and Cd Estimate (Blue) against Time (hours)")
plt.plot(Ts, Cd0*(1+xs[:,6]), 'r')
plt.plot(Ts, Cd0*(1+xs_kf[:,6]), 'b')
#plt.plot(Ts, xs[:,6], 'r')
#plt.plot(Ts, xs_kf[:,6], 'b')
plt.ylabel("Cd")
plt.xlabel("Time (hours)")
plt.show()

We see that $C_d$ has impulsively changed from about 2 to 4 around hour three of the simulation. The estimation errors deviate outside their respective 1-sigma bound only slightly more than if $C_d$ was forced by a Gaussian. However, the estimator still performs well, despite the large change to $C_d$. This, more than anything, confirms that our process noise PSD choice of 1e-15 (for position and velocity) works well to keep the filter robust to impulsive forcing in $C_d$.

To enhance a reader's intuition, the following (simple) changes are proposed to the above code-block

> 0.) Remove the additive process noise in the simulation: i.e, change the line 

>> xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=W6) 

> to 

>> xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=0*W6). 

> This will keep the same noise realization without adding process noise to position and velocity. Keep this change for all experiments below.

<br>

> 1.) If you lower the process noise PSD (w_PSD) to 1e-18 (but keep $C_d$ impulsive with $\alpha=1.3$), the estimation errors begin to oscillate due to $C_d$'s impulsive nature.

<br>

> 2.) If you lower the process noise PSD (w_PSD) to 1e-18 and change $C_d$'s distribution to $\alpha=1.0$ (i.e, Cauchy distributed), the impulse becomes larger. Large oscillations in the estimation errors will appear.

<br>

> 2.) If you repeat step 2, but increase the variable "sf_boost" to say, 400, you will remove most of the oscillations. Moreover, you will track the jump in $C_d$. You can try this for $\alpha=1.3$, or any alpha value for that matter.

<br>

> 4.) If you lower the process noise PSD (w_PSD) to 1e-18 and change $C_d$'s distribution back to Gaussian, no oscillations are seen and the estimator performs quite well.

This ends tutorial 4. Well done! Hopefully, you have developed some intuition with the GMAT API, and can apply it to your own estimation problems. Happy programming!